In [27]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from scipy import interp
#from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import FaceRecognition
from time import time
import random
#from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [21]:
faces_db = fetch_lfw_people(min_faces_per_person= 20, resize=0.6)
print("Data Loaded")
no_of_samples, h, w = faces_db.images.shape
print("no of samples: %d" % no_of_samples)

Data Loaded
no of samples: 3023


In [22]:
X = faces_db.data
y = faces_db.target

In [23]:
X.shape

(3023, 4200)

In [24]:
y.shape

(3023,)

In [25]:
max(y)

61

In [26]:
y

array([61, 25,  9, ..., 14, 15, 14])

In [30]:
lb = LabelBinarizer()
yBin = lb.fit_transform(y)

In [31]:
yBin.shape

(3023, 62)

In [32]:
yBin[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [36]:
yBin.shape[0]

3023

In [37]:
yBin.shape[1]

62

In [39]:
n_classes = yBin.shape[1]#62 y.shape[0] returns 1867 the no of rows
target_names = faces_db.target_names
X_train, X_test, y_train, y_test = train_test_split(X, yBin, test_size=0.2, random_state=42)

In [44]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2418, 4200)
(605, 4200)
(2418, 62)
(605, 62)


In [43]:
no_of_components = 800
print("Extracting the top %d eigenfaces from %d faces" % (no_of_components, X_train.shape[0]))
pca = PCA(n_components=no_of_components,svd_solver='randomized', whiten=True).fit(X_train)
#n_components == min(n_samples, n_features)
eigen_faces = pca.components_ 
print(sum(pca.explained_variance_ratio_))#0.99612

Extracting the top 800 eigenfaces from 2418 faces
0.996124531345


In [45]:
no_eigen_faces,no_features = eigen_faces.shape#150,1850
eigen_faces_lowd = pca.transform(eigen_faces)#dimensionality reduction applied
#eigen_faces_lowd = eigen_faces.dot(eigen_faces.transpose())
weight_matrix = pca.transform(X_train)
#weight_matrix = X_train.dot(eigen_faces.transpose())
print(eigen_faces.shape)
print(eigen_faces_lowd.shape)#150 dimension and 150 number of eigen faces
print(weight_matrix.shape)

(800, 4200)
(800, 800)
(2418, 800)


In [46]:
a = weight_matrix.sum(0)#summing all the rows
a = a/X_train.shape[0]
max_eigen_index = np.argmax(a)#index of max element in an array
min_eigen_index = np.argmin(a)
print(max_eigen_index, min_eigen_index)


774 797
